In [12]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.jmt.*
import jline.solvers.mva.*
import jline.util.matrix.*

In [13]:
// Fork-Join network example 4
val model = Network("model")

// Create network
val source = Source(model, "Source")
val queue1 = Queue(model, "Queue1", SchedStrategy.PS)
val queue2 = Queue(model, "Queue2", SchedStrategy.PS)
val queue3 = Queue(model, "Queue3", SchedStrategy.PS)
val fork = Fork(model, "Fork")
val sink = Sink(model, "Sink")

val jobclass1 = OpenClass(model, "class1")

In [14]:
// Service configurations
source.setArrival(jobclass1, Exp(0.5))
queue1.setService(jobclass1, Exp(1.0))
queue2.setService(jobclass1, Exp(2.0))
queue3.setService(jobclass1, Exp(3.0))

In [15]:
// Pure fork routing (no join)
val P = model.initRoutingMatrix()
P.set(jobclass1, jobclass1, source, fork, 1.0)
P.set(jobclass1, jobclass1, fork, queue1, 1.0)
P.set(jobclass1, jobclass1, fork, queue2, 1.0)
P.set(jobclass1, jobclass1, fork, queue3, 1.0)
P.set(jobclass1, jobclass1, queue1, sink, 1.0)
P.set(jobclass1, jobclass1, queue2, sink, 1.0)
P.set(jobclass1, jobclass1, queue3, sink, 1.0)
model.link(P)

In [ ]:
import jline.solvers.NetworkSolver

// Solve with multiple methods
val solvers = mutableListOf<NetworkSolver>()

solvers.add(JMT(model, "seed", 23000))
solvers.add(MVA(model))

for ((i, solver) in solvers.withIndex()) {
    println("SOLVER ${i+1}: ${solver.name.replace("Solver", "")}")
    try {
        val avgTable = solver.avgTable
        avgTable.print()
    } catch (e: Exception) {
        println("Error: ${e.message}")
    }
}